In [1]:
from astropy.coordinates import SkyCoord
import gPhoton
from gPhoton.MCUtils import print_inline
from gPhoton import PhotonPipe
import numpy as np
import os
import requests

In [2]:
# Verify that the modified version of PhotonPipe is in use:
if not gPhoton.__version__ == '1.28.9_nomask':
    raise RuntimeError('Incorrect version of PhotonPipe. You must use the branch'
                      '"1.28.9_nomask" available in the gPhoton GitHub repository.')

In [3]:
# Import the function definitions that accompany this notebook tutorial.
nb_funcdef_file = "function_defs.py"
if os.path.isfile(nb_funcdef_file):
    from function_defs import listdir_contains, write_image, write_movie, make_lightcurve
else:
    raise IOError("Could not find function definition file '" + nb_funcdef_file + "' that goes with this notebook.")

In [4]:
# Define the sky position of UV Ceti from GAIA DR2
# Precision doesn't matter here because we'll dial it in later from the data.
hhmmss = SkyCoord('01h39m01.6334059022s', '-17d57m01.121866655s', frame='icrs')
skypos = (hhmmss.ra.degree, hhmmss.dec.degree)
print("Gaia RA, Dec (J2015.5): " + repr(skypos))
# This is closer to the position in the GALEX catalog, so we'll start with this coordinate
# (both in degrees).
skypos = (24.76279, -17.94948)
print("Using RA, Dec values of " + repr(skypos) + " degrees.")
# Best available distance to UV Ceti from GAIA DR2
parallax = 372.1631 # mas
distance = 1/(372.1631/1000) # parsecs
print('Distance: {d} +/- 0.002 pc'.format(d=round(distance,3)))
%store skypos
%store distance

Gaia RA, Dec (J2015.5): (24.756805857925826, -17.950311629626388)
Using RA, Dec values of (24.76279, -17.94948) degrees.
Distance: 2.687 +/- 0.002 pc
Stored 'skypos' (tuple)
Stored 'distance' (float)


In [5]:
# Define the data volume.
data_directory = './raw_files/'
# Check that the data volume is available, if not, create the directory path.
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
# Store the directory for use in other notebooks.
%store data_directory

Stored 'data_directory' (str)


In [6]:
# Get the locations of -raw6 and -scst files corresponding to a particular location.
# These will be downloaded and used to create the light curves we analyze.
url = "https://mastcomp.stsci.edu/portal/Mashup/MashupQuery.asmx/GalexPhotonListQueryTest?query=exec spGetRawUrls @ra={ra}, @dec={dec}, @radius={radius}&format=extjs".format(
        ra=skypos[0], dec=skypos[1], radius=10)
raw_urls = [foo[-1] for foo in gPhoton.gQuery.getArray(url)]
n_raw_urls_expected = 48
if len(raw_urls) != n_raw_urls_expected: # regression test
    raise ValueError("Expected " + str(n_raw_urls_expected) + " raw_urls but found " + str(len(raw_urls)))
# If you want to see the URLs that contain the raw data, set the print_out_urls value to True
print_out_urls = False
if print_out_urls:
    for raw_url in raw_urls:
            print(raw_url)

In [7]:
# This will download the raw files.  Note, this can take a while the first time (20 minutes or
# longer), but only needs to be done one time, future runs of this notebook will not
# re-download the files if it finds them in the directory defined in "data_directory" above.
for raw_url in raw_urls:
    filepath = '{data_directory}{filename}'.format(data_directory=data_directory, filename=raw_url.split('/')[-1])
    if not os.path.exists(filepath):
        print_inline('Downloading {url} to {filepath}'.format(url=raw_url, filepath=filepath))
        r = requests.get(raw_url)
        with open(filepath, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
print("\nAll files downloaded or found on disk.")


All files downloaded or found on disk.


In [8]:
# Regenerate the calibrated photon list files.
raw6_fuv = listdir_contains(data_directory,'fd-raw6')
for raw6 in raw6_fuv:
    if os.path.exists(raw6[:-13]+'.csv'):
        continue
    PhotonPipe.photonpipe(raw6[:-13],'FUV',raw6file=raw6,scstfile=raw6.replace('fd-raw6','scst'),verbose=2)
print_inline('Done.')

raw6_nuv = listdir_contains(data_directory,'nd-raw6')
for raw6 in raw6_nuv:
    if os.path.exists(raw6[:-13]+'.csv'):
        continue
    PhotonPipe.photonpipe(raw6[:-13],'NUV',raw6file=raw6,scstfile=raw6.replace('nd-raw6','scst'),verbose=2)
print_inline('Done.')

In [9]:
# Verify that all of the photon data exist.
photon_files = {'NUV':listdir_contains(data_directory,'nd.csv'),
                'FUV':listdir_contains(data_directory,'fd.csv')}
for band in ['NUV','FUV']:
    n_expected = 14
    if len(photon_files[band]) != n_expected:
        print(len(photon_files[band]))
        raise ValueError("Did not find expected number (" + str(n_expected) + ") of the csv files for band " + band +
                             ".  Found " + str(len(photon_files[band])) + " instead.")
    print('There are {n} {band} observations available, as expected.'.format(n=len(photon_files[band]), band=band))
    # Set the variable below to True if you want to print the names of the photon event .csv files.
    print_found_csv = False
    if print_found_csv:
        for photon_file in photon_files[band]:
            print('\t{pf}'.format(pf=photon_file.split('/')[-1]))

There are 14 NUV observations available, as expected.
There are 14 FUV observations available, as expected.


In [10]:
# Create the FITS image files.  Only the first nine visits are created, because they are the deepest exposure times,
# the tenth visit is not shown in plots.  Set "overwrite" to False to avoid re-creating FITS files that already exist.
n_visits = 9
for band in ['FUV','NUV']:
    for i in np.arange(n_visits):
        print('FITS image in {b}: Visit {i}'.format(b=band, i=i+1))
        write_image(photon_files[band][i], band, overwrite=False)

FITS image in FUV: Visit 1
FITS image in FUV: Visit 2
FITS image in FUV: Visit 3
FITS image in FUV: Visit 4
FITS image in FUV: Visit 5
FITS image in FUV: Visit 6
FITS image in FUV: Visit 7
FITS image in FUV: Visit 8
FITS image in FUV: Visit 9
FITS image in NUV: Visit 1
FITS image in NUV: Visit 2
FITS image in NUV: Visit 3
FITS image in NUV: Visit 4
FITS image in NUV: Visit 5
FITS image in NUV: Visit 6
FITS image in NUV: Visit 7
FITS image in NUV: Visit 8
FITS image in NUV: Visit 9


In [11]:
# Create light curves with 30-second sampling.  This is the initial creation of the light curves.
for i in np.arange(len(photon_files['NUV'])):
    print("Creating light curve files for visit # " + str(i+1) + ' / ' + str(len(photon_files['NUV'])) + "...")
    lc_nuv = make_lightcurve(photon_files['NUV'][i], 'NUV', stepsz=30., skypos=skypos, makefile=False)
    try:
        lc_fuv = make_lightcurve(photon_files['NUV'][i].replace('nd','fd'), 'FUV', stepsz=30.,
                             skypos=skypos, fixed_t0=lc_nuv['t0'].min(), makefile=False)
    except:
        print('Skipping: No FUV data?')
        continue
    tranges = zip(lc_fuv['t0'], lc_fuv['t1'])
    write_movie(photon_files['NUV'][i].replace('nd','fd'), 'FUV', tranges)
    tranges = zip(lc_nuv['t0'], lc_nuv['t1'])
    write_movie(photon_files['NUV'][i], 'NUV', tranges)

Creating light curve files for visit # 1 / 14...
Creating light curve files for visit # 2 / 14...            
Creating light curve files for visit # 3 / 14...            
Creating light curve files for visit # 4 / 14...            
Creating light curve files for visit # 5 / 14...            
Creating light curve files for visit # 6 / 14...            
Creating light curve files for visit # 7 / 14...            
Creating light curve files for visit # 8 / 14...            
Creating light curve files for visit # 9 / 14...            
Creating light curve files for visit # 10 / 14...           
Creating light curve files for visit # 11 / 14...           


/Users/fleming/Work/GALEX/gPhoton/SCIENCE/FLARE_STARS/UV_CETI/gfcat/src/uvceti/gPhoton/MCUtils.py:349: RuntimeWarning: invalid value encountered in arcsin
  r = 2*np.arcsin(np.sqrt(a))
/Users/fleming/Work/GALEX/gPhoton/SCIENCE/FLARE_STARS/UV_CETI/gfcat/src/uvceti/function_defs.py:224: RuntimeWarning: invalid value encountered in less_equal
  ix = np.where((angsep <= aper) & (np.isfinite(angsep)) &
/Users/fleming/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/fleming/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Recentering aperture on [nan, nan]


/Users/fleming/Work/GALEX/gPhoton/SCIENCE/FLARE_STARS/UV_CETI/gfcat/src/uvceti/function_defs.py:236: RuntimeWarning: invalid value encountered in less_equal
  ix = np.where((angsep <= aper) & (np.isfinite(angsep)) &


Skipping: No FUV data?
Creating light curve files for visit # 12 / 14...
Recentering aperture on [nan, nan]0s.csv                    
Skipping: No FUV data?
Creating light curve files for visit # 13 / 14...
Creating light curve files for visit # 14 / 14...           
